<a href="https://colab.research.google.com/github/hejazn86/NLP_IMBD_reviews/blob/master/main_NLP_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)
print(tfds.__version__)

2.2.0
2.1.0


In [2]:
#importing the data from tensorflow_datasets
import numpy as np

imbd, info = tfds.load("imdb_reviews", with_info= True, as_supervised=True)
train_set, test_set = imbd['train'], imbd['test']

print(train_set)

train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

for s, l in train_set :
  train_sentences.append(s.numpy().decode('utf8'))
  train_labels.append(l.numpy())

for s, l in test_set:
  test_sentences.append(s.numpy().decode('utf8'))
  test_labels.append(l.numpy())


train_sentences = np.array(train_sentences)
test_sentences = np.array(test_sentences)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
print(train_labels.shape)
print(train_sentences[0])

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJMFXMS/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJMFXMS/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJMFXMS/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>
(25000,)
This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words= 10000
embadding_output_dim = 15

tokenizer = Tokenizer(num_words= num_words, oov_token = 'unkw')
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded_sequences = pad_sequences(train_sequences, maxlen=150, padding='post', truncating='post')

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded_sequences = pad_sequences(test_sequences, maxlen=150, padding='post', truncating='post')

In [11]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(10000, embadding_output_dim, input_length=150),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['acc'])

In [12]:
history = model.fit(train_padded_sequences, train_labels, epochs=10, validation_data=(test_padded_sequences, test_labels))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5029 - acc: 0.7813 - val_loss: 0.3652 - val_acc: 0.8419
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2899 - acc: 0.8830 - val_loss: 0.3459 - val_acc: 0.8498
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2364 - acc: 0.9086 - val_loss: 0.3663 - val_acc: 0.8436
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2018 - acc: 0.9244 - val_loss: 0.4044 - val_acc: 0.8328
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1769 - acc: 0.9352 - val_loss: 0.4363 - val_acc: 0.8314
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1566 - acc: 0.9438 - val_loss: 0.4769 - val_acc: 0.8266
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1385 - acc: 0.9530 - val_loss: 0.5189 - val_acc: 0.8232
Epoch 8/10
782/782 [==============================] - 4s 5ms/step - loss: 0.

In [13]:
val_loss, val_acc = model.evaluate(test_padded_sequences, test_labels)

import matplotlib.pyplot as plt
 
epochs = range(len(history.history['acc'])


plt.subplot(121).plot(epochs, history.history['loss'], 'b')
plt.subplot(121).plot(epochs, history.history['val_loss'], 'r')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.grid(True)

plt.subplot(122).plot(epochs, history.history['acc'], 'b')
plt.subplot(122).plot(epochs, history.history['val_acc'], 'r')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.grid(True)
plt.show()

SyntaxError: ignored